<div class="alert alert-danger">
    <p><strong>NOTE:</strong></p>
    <ol>
        <li><strong>restart the kernel</strong> (in the menubar, select <strong>Kernel → Restart</strong>)
        <li><strong>run all cells</strong> (in the menubar, select <strong>Cell → Run All</strong>)</li>
    </ol>
    <p>Make sure to complete every cell that states "<strong><TT>YOUR CODE HERE</TT></strong>" or "<strong><TT>YOUR ANSWER HERE</TT></strong>".</p>
</div>

---

<div class="alert alert-info">
    <h3 style="text-decoration:underline;">Version 2: Changes</h3>
    <p>Please view the blue bubbles (similar to the one encapsulating this text) with the heading <strong>Update</strong> for revised instructions, clarifications, or added details.</p>
</div>

---



This task involves **classification**.

You will classify a set of images using simple implementations of classifiers (linear, SVM, and decision tree classifiers) and an ensemble of the three classifiers. Each image will consist of either a cat or a dog. The classifier must correctly label each image as either an image of a *cat* or as an image of a *dog*.
![alt text](https://storage.googleapis.com/tfds-data/visualization/fig/cats_vs_dogs-4.0.0.png "Dogs & Cats Image")

We will then create a more diverse and obfuscated set of test images to evaluate the robustness of the models. Such an obfuscated image could appear as follows:

![alt text](https://www.tensorflow.org/tutorials/generative/deepdream_files/output_tEfd00rr0j8Z_0.png "DeepDreamed Image")

# Preliminaries & Dependencies

You will need to install the following **Python** packages to run

    pip install matplotlib numpy
    pip install sklearn
    pip install tensorflow tensorflow-datasets

# Overview

We want to determine the difference in a model's performance when evaluated with data that has been altered or obfusctaed to make the task more difficult to perform.

The theme is *composition* ("*the principle of progressive disclosure of complexity*", to borrow from [Keras](https://keras.io/about)). We will extend and build upon the ideas we have previously explored.


# Objectives

Access a *real-world* dataset (e.g., used in kaggle.com competitions, etc.) rather than a *toy* dataset.

Generate data from existing, actual data (contrast this to generating synthetic data).

Download and implement already trained models.

Evaluate the robustness of a model by testing it on data that is designed to be more *complex* than the training data.

Develop and evaluate ensembles of models.

# Acquire Data

We will use the [Cats and Dogs dataset](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs) (size: 786.7 Mb).
The dataset consists of images (the input) and labels, either *cat* or *dog* (the output).
Information on the dataset is [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765).

Our **goal** is to predict which of two labels (*cat* or *dog*) to apply to an image. This is a binary classification task (i.e., *two categories, two labels, two classes*, etc.).

Downlading a dataset (https://www.tensorflow.org/tutorials/images/data_augmentation#apply_augmentation_to_a_dataset):

In [ ]:
(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

f, y = (features, output)


# Transforming Data

We will essentially **augment** our dataset. Augmenting data "*increases the diversity of your training set by applying random (but realistic) transformations such as image rotation*" (https://www.tensorflow.org/tutorials/images/data_augmentation).

In our case we are *not* augmenting the data to collect more diverse **training data** but to *evaluate* the models on more diverse/complex **testing data**. Thus we will **only** transform the **test data**.



## Examples Of Data Augmentation

We will be doing something similar to:
https://www.tensorflow.org/datasets/catalog/moving_mnist
where the MNIST hadwritten digit image dataset was transformed into a video of animated/moving handwritten digits (click *Display Examples* to view the videos).

Augmenting a dataset of flower images:
https://www.tensorflow.org/tutorials/images/data_augmentation#using_tfimage
which uses warping and colour transformations on flower pictures.

## Example Image Transformations

<div class="alert alert-danger">
    <h4>NOTE</h4>
    This resize_and_rescale function example is provided for your information. We will <strong>not</strong> be using the code in this example.
</div>

The code (taken from **Tensorflow**'s explanation for [transforming an image dataset](https://www.tensorflow.org/tutorials/images/data_augmentation#apply_augmentation_to_a_dataset)) is to show how simple the image processing step is.


Create a function that resizes and rescales the images (so as to "*unify the size and scale of images in the dataset*"):

In [ ]:
def resize_and_rescale(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = (image / 255.0)
    return image, label

<div class="alert alert-danger">
    <h4>NOTE</h4>
    This example (the augment function) is provided for your information. We will <strong>not</strong> be using the code in this example.
</div>

Create an *augment* function that applies random transformations to an image:

In [ ]:
def augment(image_label, seed):
    image, label = image_label
    image, label = resize_and_rescale(image, label)
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
    
    # Make a new seed
    new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
    
    # Random crop back to the original size
    image = tf.image.stateless_random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
    
    # Random brightness
    image = tf.image.stateless_random_brightness(image, max_delta=0.5, seed=new_seed)
    
    image = tf.clip_by_value(image, 0, 1)
    
    return image, label

# Stylistic Transformation

We will transform the image data *stylistically* by processing images with [**DeepDream**](https://www.tensorflow.org/tutorials/generative/deepdream).

For example, this image of a labrador:

![alt text](https://www.tensorflow.org/tutorials/generative/deepdream_files/output_Y5BPgc8NNbG0_0.png "Original Image")

transforms into the following image after being processed by **DeepDream**:

![alt text](https://www.tensorflow.org/tutorials/generative/deepdream_files/output_tEfd00rr0j8Z_0.png "DeepDreamed Image")

A description and tutorial implementation of **Deep Dream** (aka **Inceptionism**) is [here](https://www.tensorflow.org/tutorials/generative/deepdream). Another type of transformation we could have applied to the images was a [Style Transfer](https://www.tensorflow.org/tutorials/generative/style_transfer).

<div class="alert alert-danger">
    <h4>NOTE</h4>
    We are <strong>not</strong> concerned with the details of <strong>DeepDream</strong>. We will use it as an off-the-shelf component in our system to augment our image dataset for evaluating how a model performs on the <strong>DeepDreamed</strong> images.
</div>

<div class="alert alert-success">
    <h4>Code To Execute Begins Here</h4>
</div>

## Imports

Import the following libraries.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image
from tensorflow.keras.preprocessing import image

## Image Preparation

**DeepDream**'s image preparation (code is from https://www.tensorflow.org/tutorials/generative/deepdream#choose_an_image_to_dream-ify):


In [ ]:
# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
    name = url.split('/')[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

# Normalize an image
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)

# Display an image
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))

# image we will process
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'

# Downsizing the image makes it easier to work with.
original_img = download(url, max_dim=500)
show(original_img)
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

## Prepare Feature Extraction Model

We we do not need to be concerned with the details (we will use the code as it is *off-the-shelf*), but feel free to play with the **layers** to see what effect they have on an image.

The explanation of the following code is provided FYI and is taken from https://www.tensorflow.org/tutorials/generative/deepdream#prepare_the_feature_extraction_model.

> ...the layers of interest are those where the convolutions are concatenated. There are 11 of these layers in InceptionV3, named 'mixed0' though 'mixed10'. Using different layers will result in different dream-like images. Deeper layers respond to higher-level features (such as eyes and faces), while earlier layers respond to simpler features (such as edges, shapes, and textures). Feel free to experiment with the layers selected below, but keep in mind that deeper layers (those with a higher index) will take longer to train on since the gradient computation is deeper.

> The complexity of the features incorporated depends on layers chosen by you, i.e, lower layers produce strokes or simple patterns, while deeper layers give sophisticated features in images, or even whole objects.



In [ ]:
# Maximize the activations of these layers
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

## Calculate Loss

The following is from https://www.tensorflow.org/tutorials/generative/deepdream#calculate_loss.
Again, we aren't concerned with the details and will use the code as it is.

> The **loss** is the sum of the activations in the chosen layers. The loss is normalized at each layer so the contribution from larger layers does not outweigh smaller layers. Normally, *loss is a quantity you wish to minimize via gradient descent*. In **DeepDream**, you will *maximize this loss via gradient ascent*.

In [ ]:
def calc_loss(img, model):
    # Pass forward the image through the model to retrieve the activations.
    # Converts the image into a batch of size 1.
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    if len(layer_activations) == 1:
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return tf.reduce_sum(losses)

## Gradient Ascent

The following is from https://www.tensorflow.org/tutorials/generative/deepdream#gradient_ascent.
Again, we aren't concerned with the details and will use the code as it is.

> After the loss for the chosen layers is calculated, calculate the gradients with respect to the image and add them to the original image.
Adding the gradients to the image **enhances the patterns seen by the neural network**. At each step, we create an image that **increasingly excites the activations of certain layers** in the network.

> The method that does this is wrapped in a `tf.function` for performance. It uses an `input_signature` to ensure that the function is not retraced for different image sizes or `steps/step_size` values.

In [ ]:
class DeepDream(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(
        input_signature=(
            tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
            tf.TensorSpec(shape=[], dtype=tf.int32),
            tf.TensorSpec(shape=[], dtype=tf.float32),)
    )
    def __call__(self, img, steps, step_size):
        print("Tracing")
        loss = tf.constant(0.0)
        for n in tf.range(steps):
            with tf.GradientTape() as tape:
                # This needs gradients relative to `img`
                # `GradientTape` only watches `tf.Variable`s by default
                tape.watch(img)
                loss = calc_loss(img, self.model)

            # Calculate the gradient of the loss with respect to the pixels of the input image.
            gradients = tape.gradient(loss, img)

            # Normalize the gradients.
            gradients /= tf.math.reduce_std(gradients) + 1e-8 

            # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
            # You can update the image by directly adding the gradients (because they're the same shape!)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)

        return loss, img

deepdream = DeepDream(dream_model)

## DeepDream: Main Loop

From https://www.tensorflow.org/tutorials/generative/deepdream#main_loop.
Again, we are not concerned with the details and will use the code as it is.

In [ ]:
def run_deep_dream_simple(img, steps=100, step_size=0.01):
    # Convert from uint8 to the range expected by the model.
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    
    step_size = tf.convert_to_tensor(step_size)
    steps_remaining = steps
    step = 0
    
    while steps_remaining:
        if steps_remaining>100:
            run_steps = tf.constant(100)
        else:
            run_steps = tf.constant(steps_remaining)
        steps_remaining -= run_steps
        step += run_steps

        loss, img = deepdream(img, run_steps, tf.constant(step_size))

        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Step {}, loss {}".format(step, loss))

    result = deprocess(img)
    display.clear_output(wait=True)
    show(result)

    return result

Process an image and view the result. This should process the labrador image mentioned earlier and display the **DeepDreamed** image to the screen.

In [ ]:
dream_img = run_deep_dream_simple(img=original_img, steps=100, step_size=0.01)


# Generate New Image Data Via DeepDream

<div class="alert alert-info">
    <h3 style="text-decoration:underline;">UPDATE</h3>
    <p>The <strong>Cats & Dogs</strong> dataset consists of 20,000 images, which is too many to process by <strong>DeepDream</strong> (as well as taking too long to train the simpler classification models).</br>
    Possible approach: use only 1,000 images from the original dataset (800 training, 200 testing).</p>
    <p>One method to read the <strong>Cats & Dogs</strong> data is by using the <strong>Tensorflow-Datasets</strong> module:</p>
</div>

In [ ]:
import tensorflow_datasets as tfds

(train_dataset, test_dataset), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

<div class="alert alert-info">
    <h3 style="text-decoration:underline;">UPDATE</h3>
    <p>Load <strong>Cats & Dogs</strong> dataset, reshape/resize, and save to a new file.</p>
    <p><strong>Keras</strong> provides image preprocessing tools we can use.</p>
</div>

In [ ]:
from os import listdir
from numpy import asarray
from numpy import save
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# location of downloaded Dogs vs. Cats image dataset on my computer
#folder = '/Volumes/WDBook/dogs-vs-cats/train/'
bookmark
# subset of original dataset consisting of 1,000 images
folder = '/Volumes/WDBook/dogs-vs-cats/train-1000/'

# subset of original dataset consisting of 10,000 images
#folder = '/Volumes/WDBook/dogs-vs-cats/train-10000/'

<div class="alert alert-info">
    <h4 style="text-decoration:underline;">UPDATE</h4>
    <p>Because the images in the dataset are of many different dimensions, we will need to resize the images so they are all 200x200 pixels.</br>
    This transformation will result in distorting, stretching, etc. every image to conform to a 200x200 image.</p>
    <p></p>
    <p>Sample code to resize/reshape images:</p>
</div>

In [ ]:
# from keras.preprocessing.image import load_img
# resized_photo = load_img("/path/to/original_image_file", target_size=(200, 200))

<div class="alert alert-info">
    <h4 style="text-decoration:underline;">UPDATE</h4>
    Convert the resized image to a Python array:
</div>

In [ ]:
# resized_photo = img_to_array(resized_photo)

<div class="alert alert-info">
    <h4 style="text-decoration:underline;">UPDATE</h4>
    <p>After resizing the images, calculate the amount of memory (RAM) the computer will require to process the entire <strong>Cats & Dogs</strong> image dataset.</p>
    <h4 style="text-decoration:underline;">Size Of Dataset With 20,000 Images</h4>
    <p>Using all <strong>20,000 images</strong> of the original dataset:</br>
    20,000 images <strong>x</strong> 200 x 200 x 3 pixels per image</br>
     = 2,400,000,000 32-bit pixels</br>
     = 76,800,000,000 bits</br>
     = 9,600,000,000 bytes (conversion: 8 bits in 1 byte)</br>
     = <strong>9.6 Gbytes</strong></p>
    <p></br></p>
    <h4 style="text-decoration:underline;">Size Of Dataset With 1,000 Images</h4>
    <p>Using only <strong>1,000 resized images</strong> from the original dataset:</br>
    1,000 images <strong>x</strong> 200 x 200 x 3 pixels per image</br>
     = 120,000,000 32-bit pixels = 3,840,000,000 bits = 480,000,000 bytes</br>
     = <strong>0.48 Gb</strong></p>
    <p></br></p>
    <h4 style="text-decoration:underline;">Size Of Dataset With 10,000 Images</h4>
    <p>Using <strong>10,000 resized images</strong> from the original dataset:</br>
    10,000 images <strong>x</strong> 200 x 200 x 3 pixels per image</br>
     = 1,200,000,000 32-bit pixels = 38,400,000,000 bits = 4,800,000,000 bytes</br>
     = <strong>4.8 Gb</strong></p>
</div>

<div class="alert alert-danger">
    <h3 style="text-decoration:underline;">NOTE</h3>
    <p>If the dataset is larger than the amount of available <strong>RAM</strong>, then Jupyterlab will display a message similar to:</br>
    "The kernel for A2.ipynb appears to have died. It will restart automatically."</p>
    <p></p>
    Successful execution of the below code will display something similar to:</br>
    (1000, 200, 200, 3) (1000,)</p>
</div>

In [ ]:
photos, labels = list(), list()

# processing every file in a folder
for file in listdir(folder):
	# determine label of image from filename (cat = 1, dog = 0)
	output = 0.0 
	if file.startswith('cat'):
		output = 1.0
	
	photo = load_img(folder + file, target_size=(200, 200))
    
	# convert image to a Python array
	photo = img_to_array(photo)
	
    # store converted image & its corresponding output label
	photos.append(photo)
	labels.append(output)

# converts list of images to a Python array
photos = asarray(photos)
labels = asarray(labels)

# print(photos.shape, labels.shape)

# save resized photos to avoid having to repeat the above process
save('/Volumes/WDBook/dogs-vs-cats/dogs_vs_cats_photos.npy', photos)
save('/Volumes/WDBook/dogs-vs-cats/dogs_vs_cats_labels.npy', labels)

<div class="alert alert-info">
    <h3 style="text-decoration:underline;">UPDATE</h3>
    Load the saved reshaped images then confirm their shape (i.e., dimension).
</div>

In [ ]:
from numpy import load

photos = load('/Volumes/WDBook/dogs-vs-cats/dogs_vs_cats_photos.npy')
labels = load('/Volumes/WDBook/dogs-vs-cats/dogs_vs_cats_labels.npy')

# print(photos.shape, labels.shape)

### Analysis Of Dataset

Provide empirical information about the Cats & Dogs dataset.

<div class="alert alert-success">
    <h4>BONUS</h4>
    Write code to provide empirical information about the <strong>Cats & Dogs</strong> dataset. Use visual elements where possible.
</div>

The images in the dataset are in multitudes of dimensions, 
they use different backgrounds, have different contrasts of colors, 
Some images are blur and hazy

![image.png](attachment:20171de2-b037-44a7-b923-8d4ee75fffb2.png)

while some are extremely sharp and defined.
A weird thing is that some images feature people which I am not sure how it affects the training of the models

![image.png](attachment:5e92bca4-ed18-48ad-bf90-5b300d106439.png)
![image.png](attachment:8a8e81cd-af1e-43ba-841d-3c810c45f732.png)

Some images were overexposed, could work well as a training or testing datasets especially if there was another process like deepdreaming to overexpose the images and test the predcition on them

![image.png](attachment:139d1c58-8ed8-4bb1-8114-9790fd6ea368.png)


Dogs have less patchy patterns and have consistent colors, which might help the algorithm. Cats however, run a lot of different patterns which can range in a lot of colors. 
Cats have better defined whiskers as well.

![image.png](attachment:81721f9f-5ac6-4cf0-af45-4dede87e3e80.png)
![image.png](attachment:3053c9e9-8d58-433f-b719-4e21c1091584.png)

### Create Testing & Training Datasets

Separate the **Cats & Dogs** dataset into a test set and a training set.

<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code to separate the data into a test set and a training set below.
</div>

In [ ]:
from os import listdir
from numpy import asarray
from numpy import save
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# location of downloaded Dogs vs. Cats image dataset on my computer
#folder = '/Volumes/WDBook/dogs-vs-cats/train/'

# subset of original dataset consisting of 1,000 images
folderCats = 'D:/Development/CMPT 310 CodingQuiz#2/Cats'
folderDogs = 'D:/Development/CMPT 310 CodingQuiz#2/Dogs'

photos, labels = list(), list()
fileNumCats = 0
fileNumDogs = 501
# processing every file in a folder
for file in listdir(folderCats):
   
    fileStr = str(fileNumCats)
   
    output = 1.0

    photo = load_img(folderCats + '/' + fileStr + '.jpg', target_size=(200, 200))
    fileNumCats = fileNumCats+1
    
    # convert image to a Python array
    photo = img_to_array(photo)
    
    # store converted image & its corresponding output label
    photos.append(photo)
    labels.append(output)
    
    
for file in listdir(folderDogs):
   
    fileStr = str(fileNumDogs)
	# determine label of image from filename (cat = 1, dog = 0)
    output = 0.0

    photo = load_img(folderDogs + '/' + fileStr + '.jpg', target_size=(200, 200))
    fileNumDogs = fileNumDogs+1
    
    # convert image to a Python array
    photo = img_to_array(photo)
    
    # store converted image & its corresponding output label
    photos.append(photo)
    labels.append(output)

# converts list of images to a Python array
photos = asarray(photos)
labels = asarray(labels)


# print(photos.shape, labels.shape)

# save resized photos to avoid having to repeat the above process
save('D:/Development/CMPT 310 Presentation/Images.npy', photos)
save('D:/Development/CMPT 310 Presentation/Labels.npy', labels)

from sklearn.model_selection import train_test_split
img_train, img_test, label_train, label_test =  train_test_split(photos, labels, test_size=0.2, random_state=1)


### Process Testing Dataset Via DeepDream 

Create a **DeepDreamed Test Set** by processing the original test set via **DeepDream**. Display a few of the resulting **DeepDreamed** images.

<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code that processes images via <strong>DeepDream</strong> below.
</div>

In [ ]:
# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
    name = url.split('/')[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

# Normalize an image
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tf.cast(img, tf.uint8)
def show(img):
    display.display(PIL.Image.fromarray(np.array(img)))
    
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

# Maximize the activations of these layers
names = ['mixed2', 'mixed4']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

def calc_loss(img, model):
    # Pass forward the image through the model to retrieve the activations.
    # Converts the image into a batch of size 1.
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    if len(layer_activations) == 1:
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return tf.reduce_sum(losses)

class DeepDream(tf.Module):
    def __init__(self, model):
        self.model = model

    @tf.function(
        input_signature=(
            tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
            tf.TensorSpec(shape=[], dtype=tf.int32),
            tf.TensorSpec(shape=[], dtype=tf.float32),)
    )
    def __call__(self, img, steps, step_size):
        print("Tracing")
        loss = tf.constant(0.0)
        for n in tf.range(steps):
            with tf.GradientTape() as tape:
                # This needs gradients relative to `img`
                # `GradientTape` only watches `tf.Variable`s by default
                tape.watch(img)
                loss = calc_loss(img, self.model)

            # Calculate the gradient of the loss with respect to the pixels of the input image.
            gradients = tape.gradient(loss, img)

            # Normalize the gradients.
            gradients /= tf.math.reduce_std(gradients) + 1e-8 

            # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
            # You can update the image by directly adding the gradients (because they're the same shape!)
            img = img + gradients*step_size
            img = tf.clip_by_value(img, -1, 1)

        return loss, img

deepdream = DeepDream(dream_model)

def run_deep_dream_simple(img, steps=100, step_size=0.01):
    # Convert from uint8 to the range expected by the model.
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    
    step_size = tf.convert_to_tensor(step_size)
    steps_remaining = steps
    step = 0
    
    while steps_remaining:
        if steps_remaining>100:
            run_steps = tf.constant(100)
        else:
            run_steps = tf.constant(steps_remaining)
        steps_remaining -= run_steps
        step += run_steps

        loss, img = deepdream(img, run_steps, tf.constant(step_size))

        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Step {}, loss {}".format(step, loss))

    result = deprocess(img)
    display.clear_output(wait=True)
    show(result)

    return result


dream_imgs = list()

for x in range(len(img_test)):
    dream_img = run_deep_dream_simple(img = img_test[x], steps=100, step_size=0.01)
    dream_imgs.append(dream_img)
    print(x)
    if(x % 50 == 0):
        show(dream_img)

dream_imgs = asarray(dream_imgs)
save(folder + '/' + 'deepDreamed', dream_imgs)
    
# RAM TO process all the images in subset : 480960000 or 0.48gb

### Style Transfer 

<div class="alert alert-success">
    <p>Process the test set using <a href="https://www.tensorflow.org/tutorials/generative/style_transfer">Style Transfer</a>.</p>
    <p>Display a few images processed using <strong>Style Transfer</strong>.</p>
</div>

import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import cv2

model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

from os import listdir
for file in listdir('D:/Development/CMPT 310 Presentation/Style/'):

    content_image = load_image('D:/Development/CMPT 310 Presentation/Style/' + file)
    style_image = load_image('pattern.jfif')

    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]


    plt.imshow(np.squeeze(stylized_image))
    plt.show()

![image.png](attachment:2453462c-873d-4bfd-98ad-f0a2d04a8079.png)


@Inspired from: https://www.youtube.com/watch?v=bFeltWvzZpQ

# Evaluating Individual Models 

The following classification models are to be evaluated (default parameters can be used in both models):
* [SVM classifier](https://scikit-learn.org/stable/modules/svm.html#classification) 
* [decision tree classifier](https://scikit-learn.org/stable/modules/tree.html#classification)


Models will be *trained* on the **training data**.

Two *separate evaluations* will be performed:
* models will be evaluated on the **original test data**
* models will be evaluated on the **transformed test data**


<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code below to create and evaluate the classifiers.
</div>

<div class="alert alert-info">
    <h4>TIP</h4>
    Classifiers take two arrays as input:</br>
    <strong>array X</strong> of shape (number_of_samples, number_of_features) containing the training samples feature data</br>
    <strong>array y</strong> of class labels (strings or integers) of shape (number_of_samples)</p>
    <p></p>
    <p>print(photos.shape, labels.shape)</br>
    num_samples = labels.shape[0]</br>
    x = np.reshape(photos, (num_samples, -1))</p>
</div>

In [ ]:
from sklearn import svm



svm = svm.SVC()

print(len(img_test))

img_train = img_train.reshape(801,3*200*200)
img_test = img_test.reshape(201,3*200*200)

svm.fit(img_train,label_train)
prediction = svm.predict (img_test)
from sklearn.metrics import accuracy_score
accuracy_score(prediction, label_test)
#0.5771144278606966

Dream = load('D:/Development/CMPT 310 Presentation/deepDreamed.npy')
Dream = Dream.reshape(201,3*200*200)
predictionDream = svm.predict(Dream)

accuracy_score(predictionDream, label_test)
#0.46766169154228854

#SVM OVER----

from sklearn.tree import DecisionTreeClassifier
DecTree = DecisionTreeClassifier(random_state=0)
DecTree.fit(img_train,label_train)
predictionDecTree = DecTree.predict (img_test)
#0.4925373134328358
accuracy_score(predictionDecTree, label_test)
predictionDecTreeDream = DecTree.predict(Dream)
accuracy_score(predictionDecTreeDream, label_test)
#0.5323383084577115

# Create Ensemble Models

View the lecture videos for a *brief* explanation of ensemble models. Scikit-learn provides a [concise explanation of ensembles](https://scikit-learn.org/stable/modules/ensemble.html#ensemble).

### Random Forest

The **scikit-learn** implementation of **Random Forest** "*combines classifiers by averaging their probabilistic prediction, instead of letting each classifier vote for a single class*".

Code examples for implementing a **Random Forest Classifiers** are [here](https://scikit-learn.org/stable/modules/ensemble.html#forest).
API information on **Random Forest Classifiers** is [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier).

Using **scikit-learn**, create a **Random Forest** classifier consisting of the following parameters (model parameters not specified can be left to their default values):
* number of estimators = 100



<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code to create a Random Forest classifier ensemble below.
</div>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier(n_estimators=100)
RandomForest.fit(img_train,label_train)
predictionRandomForest = RandomForest.predict (img_test)
accuracy_score(predictionRandomForest, label_test)
#0.5870646766169154
predictionRandomForestDream = RandomForest.predict (Dream)
accuracy_score(predictionRandomForestDream, label_test)
#0.46766169154228854

### Voting Ensemble 

A **Voting Ensemble** classifier "*combine conceptually different machine learning classifiers and use a majority vote (hard vote) or the average predicted probabilities (soft vote) to predict the class labels*".

Using **scikit-learn**, create two **Voting Ensemble Classifiers** (*hard voting* and *soft voting*) consisting of the models created in the previous sections (**Evaluating Individal Models**, etc.):
* SVM classifier
* decision tree classifier
* Random Forest classifier

Use the default parameters for both *hard voting* and *soft voting* classifiers.

Code examples for implementing a **Voting Ensemble Classifier** is [here](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier).
API information on **Voting Ensemble Classifier** is [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier).

<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code to create a Voting Ensemble classifier below.
</div>

In [ ]:
from sklearn import svm
svmEn = svm.SVC(probability = 1)
models = [('SVM', svmEn), ('Decision Tree', DecTree), ('Random Forest', RandomForest)]

from sklearn.ensemble import VotingClassifier
EnsembleSoft = VotingClassifier(estimators = models, voting = 'soft')
EnsembleHard = VotingClassifier(estimators = models, voting = 'hard')



NameError: name 'DecTree' is not defined

# Evaluate Ensembles

Models will be *trained* on the **training data**.

Two *separate evaluations* will be performed:
* ensemble model will be evaluated on the **original test data**
* ensemble model will be evaluated on the **transformed test data**

<div class="alert alert-danger">
    <h4>WRITE CODE</h4>
    Write the code below to evaluate the ensembles on the test data.
</div>

In [ ]:
EnsembleSoft.fit(img_train,label_train)
predictionSoft = EnsembleSoft.predict(img_test)

accuracy_score(predictionSoft, label_test)
#0.48756218905472637

EnsembleHard.fit(img_train,label_train)
predictionHard = EnsembleHard.predict(img_test)
accuracy_score(predictionHard, label_test)
#0.572139303482587

predictionSoftDream = EnsembleSoft.predict(Dream)
accuracy_score(predictionSoftDream, label_test)
#0.5323383084577115
predictionHardDream = EnsembleHard.predict(Dream)
accuracy_score(predictionHardDream, label_test)
#0.46766169154228854

# Discussion Of Results 

Discuss the results (using charts and graphs where possible).
Compare the performance of the various models.
Was the performance what you expected?

Which system performed best? Why?
Which system had the worst performance? Why?

Provide some ideas to try that could improve the performance of the models (i.e., *Future Work*).

<div class="alert alert-danger">
    <h4>DISCUSSION</h4>
    <p>Provide your <strong>Discussion Of Results</strong> below. Use visual elements where possible.</p>
    <p>Feel free to include both <em>Markdown</em> cells and <em>Code</em> cells where necessary.</p>
</div>

Compare the performance of the various models.

1. SVM Normal [0.5771144278606966] vs SVM Dream [0.46766169154228854]

2. Decision Tree Normal [0.4925373134328358] vs Decision Tree Dream [0.5323383084577115]

3. Random Forest Normal [0.5870646766169154] vs Random Forest Dream [0.46766169154228854]

4. Ensemble Soft Voting - SVM, Decision Tree, Random Forest - Normal [0.48756218905472637] , Dream [0.5323383084577115]

5. Ensemble Hard Voting - SVM, Decision Tree, Random Forest - Normal [0.572139303482587], Dream [0.46766169154228854]

Using Dreamify on the datasets might have changed the images a lot in visual sense, however, in pixels it 
changesonly by a few values here and there and there are no significant changes.

SVM normal seems to work at par with the Random Forest having best accuracy out of all.

Decision Tree Dream unexpectedly performs better on DeepDream than the normal one. 

Ensemble Hard voting decides the voting class on basis on output classes of the 3 models. An ensemble model is 
theoretically better in some cases and it works on consensus of the outputs of all 3 models it was fed. 

Ensemble Soft does not exactly outputs the class value of to the data, it gives a predicted probability function 
for the classifier. It is recommended to use only when the models are well calibrated. Hard seems to have performed 
better in overall sense, because because of SVM and Random Forest performing better
on the normal dataset as individual models, having low scores in the transformed section would be due 
SVM and Random Forest performing badly on the Deep Dream. It basically took the average of all 3 models and 
computed an ensemble prediction. We can see the values are equally bad for svm and randomforest on deepdream and 
that is what Ensemble Hard shows us. 

I actually expected SVM would perform better than random forest due to its
proficiency in binary class systems while forests work better for 
multiclass problems. Random Forest would've worked even better if their were some
catergorical features.

Decision Trees can easily overfit and that's probably one of the reason's 
its normal score is so low. I am not entirely sure why decision tree performed better on dream data. I did not expect it would perform best 
on dream data when its normal score is not par with the other models that are better suited to this problem.

We can use accuracy_score since there isn't a majority class here. A confusion matrix was also an option but since it wasn't expected, I just used
the accuracy_score metric.

 I also managed to run a single layer KNN on a sequential model.
 ![image.png](attachment:64d095cd-769d-41c5-9e12-f420a969c7de.png)
 The accuracy that provided me was way better.
 
 Tutorial  https://www.youtube.com/watch?v=YrMy-BAqk8k&t=767s

I think one of the deciding factors to performance improvement was the limitation of 1000 images and the lack of better models. Our computers arent powerful enough to compress and train large amount of data. Especially in visual ML training model a lot would depend on the amount of data and the configuration of the models. The more sensitive the models are, the better they would likely perform.
It would be better to go through the data to get only the kind of images that really really improve the learning curve of the model. I also think the models we used were not especially equipped to deal with a problem like image classification



### BEGIN ANSWER

### END ANSWER